# Исследование уровня потребительской лояльности

**Цель исследования**

Определить текущий уровень потребительской лояльности (NPS) телекоммуникационной компании.

**План работы**

1. Выгрузить и отобрать необходимые данные
2. Проанализировать
3. Создать дашборд
4. Сделать презентацию

Входные даннные - **данные в SQLite с результатами опроса клиентов компании.**

In [1]:
# импортируем библиотеки

import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
# сохраняем путь БД в переменную

path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

# если путь на платформе ведёт к БД, то он становится итоговым

if os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

# создаём итоговое подключение, если путь не пустой

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
# сохраняем всю таблицу в переменную SQL запросом

query = """
SELECT *
FROM user;
"""

In [4]:
# считываем таблицу и выводим первые 3 строчки

df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,location_id,age_gr_id,tr_gr_id,lt_gr_id,nps_score
0,A001A2,2320,45.0,1.0,ANDROID,SMARTPHONE,55,5,5,8,10
1,A001WF,2344,53.0,0.0,ANDROID,SMARTPHONE,21,5,5,8,10
2,A003Q7,467,57.0,0.0,ANDROID,SMARTPHONE,28,6,9,6,10


In [5]:
# преобразовываем таблицу в csv файл

df.to_csv('telecomm_csi_tableau.csv', index=False)

In [1]:
# sql запрос для вывода необходимых столбцов

query2 = """
SELECT u.user_id,
       u.lt_day,
       (CASE
            WHEN u.lt_day > 365 THEN 'Старый'
            ELSE 'Новый'
       END) AS is_new,
       u.age,
       (CASE
            WHEN u.gender_segment = 1 THEN 'Женщина'
            ELSE 'Мужчина'
       END) AS gender_segment,
       u.os_name,
       u.cpe_type_name,
       l.country,
       l.city,
       SUBSTRING(a.title, 4) AS age_segment,
       SUBSTRING(t.title, 4) AS traffic_segment,
       SUBSTRING(ls.title, 4) AS lifetime_segment,
       u.nps_score,
       (CASE
            WHEN u.nps_score<=6 THEN 'Критик'
            WHEN u.nps_score<=8 THEN 'Нейтральный'
            ELSE 'Сторонник'
       END) AS nps_group
FROM user AS u
LEFT JOIN location l ON u.location_id=l.location_id
LEFT JOIN age_segment a ON u.age_gr_id=a.age_gr_id
LEFT JOIN traffic_segment t ON u.tr_gr_id=t.tr_gr_id
LEFT JOIN lifetime_segment ls ON u.lt_gr_id=ls.lt_gr_id;
"""

In [7]:
# сохраняем запрос в переменную и выводим первые 3 строки

data = pd.read_sql(query2, engine)
data.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,Старый,45.0,Женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,Сторонник
1,A001WF,2344,Старый,53.0,Мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,Сторонник
2,A003Q7,467,Старый,57.0,Мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,Сторонник


In [8]:
# преобразуем таблицу в csv файл

data.to_csv('telecomm_csi_tableau.csv', index=False)

# Общий вывод

Задачей проекта было определение текущего уровеня потребительской лояльности (NPS) среди клиентов из России.

**Этапы исследования**

1. Работа с данными
 - Выгрузка
 - Отбор необходимых данных
 - Сохранение
2. Анализ данных
 - Построение визуализаций
 - Выявление взаимосвязей
 - Расчёт показателя NPS
3. Создание дашборда
4. Подготовка презентации

**Выводы исследования**

1. Итоговое значение NPS -  **21.98%**
2. Наибольшее число участников среди мужчин и женщин представлено в возрастных категориях 25-34 и 35-44
3. Старых пользователей почти в 5 раз больше новых
4. Активно участвовали в опросе клиенты из Москвы и Санкт-Петербурга
5. К наиболее лояльным группам относятся:
 - Клиенты с телефонами на Android
 - Мужчины и женщины 55-64
 - Новые клиенты
6. К менее лояльным группам относятся мужчины или женщины:
 - В возрасте 35-44 лет
 - Из Москвы или Санкт-Петербурга
 - Пользуется услугами оператора менее года
 - Телефоном на базе операционной системы Android

Ссылка на дашборд: https://public.tableau.com/app/profile/yaroslav.zhdanov/viz/_16900402506730/NPS_5


Ссылка на презентацию: https://disk.yandex.ru/i/hQncyHE2eHKQBg